In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#need to add in qc and mean
# still need to add pre synaptic spike number qc. ie.e does more than one presynaptic spike happen?
# what if a postsynaptic pulse happens and it does not pass qc 

In [ ]:
def get_data_from_csv(filename):
    """Get data from a specified csv file.  Note the data arrays were written as strings with 
    funky things in them by pandas.to_csv.  Here they are pulled out and put in a dictionary as
    numpy arrays.
    Input
    -----
    filename: string
        file name or path to file
    Returns
    -------
    d: dictionary
        contains data for connections
        """
    
    data = pd.read_csv(filename)
    d={}
    d['amps'] = []
    d['spike_times_relative_to_expt'] = []
    d['post_syn_rec_id'] = []
    d['expt_start_time'] = []
    d['post_id'] = data['post_id']
    d['pre_id'] = data['pre_id']
    d['post_cre'] = data['post_cre']
    d['pre_cre'] = data['pre_cre']
    d['uid'] = data['uid']
    d['stim_freq'] = []
    d['pulse_num_within_train'] = []
#    d['qc_mask'] = []
    
    #convert weird string from cvs to arrays
    for ii in range(len(data)):
        d['amps'].append(np.fromstring(data['deconvolved_amps'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['spike_times_relative_to_expt'].append(np.fromstring(data['spike_times_relative_to_experiment'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['expt_start_time'].append(d['spike_times_relative_to_expt'][-1][0])
        d['post_syn_rec_id'].append(np.fromstring(data['post_syn_rec_id'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        d['pulse_num_within_train'].append(np.fromstring(data['pulse_num_within_train'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        #print(data['stim_freq'][ii])
        d['stim_freq'].append(np.fromstring(data['stim_freq'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        if (len(d['amps'][-1]) != len(d['spike_times_relative_to_expt'][-1])) or \
            (len(d['amps'][-1]) != len(d['post_syn_rec_id'][-1])) or \
            (len(d['amps'][-1]) != len(d['stim_freq'][-1])) or \
            (len(d['amps'][-1]) != len(d['pulse_num_within_train'][-1])):
            raise Exception('string to array conversion probably went wrong')
    return d

filename='pv_tlx_to_sst.csv'
data=get_data_from_csv(filename)

In [ ]:
def cv_of_amps_in_df(df):
    """Coefficient of variation (std/mean) of each pulse in train.  Doing
    this calculation by had here because df['amps'].std() is not working."""
    stuff=[]
    for ii, row in df.iterrows(): #/fifty['times_rel_to_train'].mean() 
        stuff.append(row['amps'])
    std=np.array(stuff).std(axis=0)
    return std/df['amps'].mean()    

In [ ]:
for ii in range(0,1): #range(len(data['uid'])):
    
    plt.figure(figsize=(15,3))
    
    # plot whole experiment
    sample=range(0,len(data['amps'][ii]))
    plt.plot(data['spike_times_relative_to_expt'][ii][sample], data['amps'][ii][sample], '.-', ms=20)
    # for ii, (p,t,a) in enumerate(zip(pulse_ids[sample], spike_times_relative_to_expt[sample], amplitudes[sample]*1e3)):
    #     plt.annotate(str(ii), xy=(t,a), textcoords='data')
    plt.ylabel('Deconvolution (arbitrary units)')
    plt.xlabel('time (s)')
    plt.title('Deconvolution Amplitude\n%.3f, pre: %s %i, post: %s %i' %(data['uid'][ii], data['pre_cre'][ii], data['pre_id'][ii], data['post_cre'][ii], data['post_id'][ii]))
    
    # create new dictionary for individual connections where trains are differentiated.
    conn_dict={}
    conn_dict['train_num'] = []
    conn_dict['post_rec_id'] = []
    conn_dict['amps'] = []
    conn_dict['times_rel_to_expt'] = []
    conn_dict['times_rel_to_train'] = []
    conn_dict['pulse_num_within_train'] = [] 
    conn_dict['stim_freq'] = []

    start_train_index = 0
    end_train_index = 0
    train_num = 1
    recording_id = data['post_syn_rec_id'][ii]
    
    # Walk though every pulse and group them acording to the recording id
    for jj in range(1, len(recording_id)):
        if (recording_id[jj-1] != recording_id[jj]):
            end_train_index = jj-1  
            #make the plot
            sample = range(start_train_index, end_train_index+1) #+1 due to range
            train_stim_freq=np.unique(data['stim_freq'][ii][sample])
            rec_id=np.unique(data['post_syn_rec_id'][ii][sample])

            # sanity checks
            if (len(train_stim_freq) != 1) or (len(rec_id) != 1):
                raise Exception('train should have one freq and one rec_id')
            
            # make data types more convenient
            train_stim_freq = np.int64(train_stim_freq[0])
            rec_id = rec_id[0]
            
            
            train_times_rel_to_expt = data['spike_times_relative_to_expt'][ii][sample]
            train_times_rel_to_train = data['spike_times_relative_to_expt'][ii][sample]-data['spike_times_relative_to_expt'][ii][start_train_index]
            train_amps = data['amps'][ii][sample]
            pulse_num_within_train = data['pulse_num_within_train'][ii][sample]
#            plt.plot(train_times_rel_to_train, train_amps, '.-', ms=20)
            start_train_index = jj
    
            #TODO: remember to eliminate trains with missing pre synaptic spikes
            if np.all(pulse_num_within_train[0:8] == np.array([1,2,3,4,5,6,7,8])):
                conn_dict['train_num'].append(train_num)
                conn_dict['post_rec_id'].append(rec_id)
                conn_dict['amps'].append(train_amps[0:8])
                conn_dict['times_rel_to_expt'].append(train_times_rel_to_expt[0:8])
                conn_dict['times_rel_to_train'].append(train_times_rel_to_train[0:8])
                conn_dict['pulse_num_within_train'].append(pulse_num_within_train[0:8])
                conn_dict['stim_freq'].append(train_stim_freq)
            
            train_num += 1
    df=pd.DataFrame(conn_dict)


    # plot individual fifty hertz trains
    plt.figure(figsize=(15,5))
    fifty=df[['times_rel_to_train', 'amps']][df['stim_freq']==50]
    for kk, row in fifty.iterrows():
        # plot each train
        plt.plot(range(1,9), row['amps'], '.-', ms=20)
    plt.plot(range(1,9), fifty['amps'].mean(), '.-k', lw=5, ms=30)
    plt.xlabel('Pulse #')
    plt.ylabel('Deconvolution (arbitrary units)')
    
    # amplitude mean versus coefficient of variation
    plt.figure(figsize=(6,6))    
    cv=cv_of_amps_in_df(fifty)
    plt.plot(fifty['amps'].mean(), cv, ms=20)
    for mm, (a, c) in enumerate(zip(fifty['amps'].mean(), cv)):
        print(a,c)
        plt.annotate(str(mm+1), (a,c))
    plt.xlabel('Mean deconvolved amplitude')
    plt.ylabel('CV deconvolved amplitude')
    plt.show()

In [ ]:
# OLD CODE: deprecate when done
# plotting if there are more than one freq 
# trains that are the same freq (note still in larger connection loop)
    stims = np.unique(conn_dict['stim_freq'])
    for stim in stims:
        plt.figure(figsize=(15,5))
        plt.title(stim)
        for jj in range(len(conn_dict['train_num'])):
            if conn_dict['stim_freq'][jj] == stim:
                plt.plot(conn_dict['times_rel_to_train'][jj][0:8], conn_dict['amps'][jj][0:8], '.-', ms=20)

In [ ]:
conn_dict['stim_freq']